---
title: "Lab 3"
format: 
    html:
        embed-resources: true
code-fold: true
echo: true
---

In [50]:
import pandas as pd
import numpy as np
xmas = pd.read_csv("https://www.dropbox.com/scl/fi/qxaslqqp5p08i1650rpc4/xmas.csv?rlkey=erdxi7jbh7pqf9fh4lv4cayp5&dl=1")
# Surpise Song (change "xmas2" --> "xmas" to sing surprise song so the same code can be used)
xmas2 = pd.read_csv("https://www.dropbox.com/scl/fi/p9x9k8xwuzs9rhp582vfy/xmas_2.csv?rlkey=kvc3j3lmyn4opcidsrhcmrof1&dl=1")
xmas.head()

,Day,Day.in.Words,Gift.Item,Verb,Adjective,Location
0,1,first,partridge,NaN,NaN,in a pear tree
1,2,second,dove,NaN,turtle,NaN
2,3,third,hen,NaN,french,NaN
3,4,fourth,bird,NaN,calling,NaN
4,5,fifth,ring,NaN,golden,NaN


In [51]:
# Function 1: pluralize_gift()
def pluralize_gift(gift):
    """
    Returns plural of a noun

  Parameters
  ----------
  gift: str
    A noun

  Return
  ------
  str
    Plural version
    """
    if "oo" in gift:
        gift = gift.replace("oo", "ee")
    elif gift[-1] == "y":
        gift = gift.replace("y","ies")
    else:
        gift = gift + "s"
    return gift


In [52]:
# Should work
pluralize_gift("goose")

# Will work if your function is vectorized!
#pluralize_gift(xmas['Gift.Item'])

'geese'

In [53]:
#Function 2: make_phrase()
def make_phrase(num, num_word, item, verb, adjective, location):
  """
  <documentation here>
  """

  ## Step 1: Replace NAs with blank strings
  verb = str(verb) if pd.notna(verb) else ""
  adjective = str(adjective) if pd.notna(adjective) else ""
  location = str(location) if pd.notna(location) else ""

  ## Step 2: If the day number is larger than 1, the gift items need pluralized!
  ### Hint: call the function you created above!
  if num > 1:
    item = pluralize_gift(item)

  ## Step 3: Figure out if a gift item starts with a vowel
  if item[0].lower() in "aeiou":
    article = "an"
  else:
    article = "a"

  ## Step 4: For the first day, if the gift item starts with a vowel, replace the day with "an" and if the gift item does not start with a vowel, replace the day with "a" (e.g. a partridge in a pear tree). If it is not the first day, use just the number word (e.g. ten lords a leap)
  if num == 1:
    lead = article
  else:
    lead = str(num_word)

  ## Step 5: Put all of the pieces together into one string and return!
  parts = [lead, adjective, item, verb, location]
  phrase = " ".join(filter(lambda s: s and s.strip(), parts))
  return phrase

In [54]:
make_phrase(12,"twelve", "drummer","drumming","","")

'twelve drummers drumming'

In [55]:
CARDINAL = {
    1:"one", 2:"two", 3:"three", 4:"four", 5:"five", 6:"six",
    7:"seven", 8:"eight", 9:"nine", 10:"ten", 11:"eleven", 12:"twelve"
}

In [56]:
xmas["Full.Phrase"] = xmas.apply(
    lambda r: make_phrase(
        r["Day"],
        CARDINAL[int(r["Day"])],
        r["Gift.Item"],
        r["Verb"],
        r["Adjective"],
        r["Location"]
    ),
    axis=1
)

In [57]:
#Function 3: sing_day()
def sing_day(dataset, num, phrase_col):
  """
  <documentation>
  """
  # Step 1: Setup the intro line
  if "Day.in.Words" in dataset.columns:
    num_word = dataset.loc[dataset["Day"] == num, "Day.in.Words"].iloc[0].lower()
  else:
    ORDINAL = {1:"first",2:"second",3:"third",4:"fourth",5:"fifth",6:"sixth",
               7:"seventh",8:"eighth",9:"ninth",10:"tenth",11:"eleventh",12:"twelfth"}
    num_word = ORDINAL[num]
  intro = "On the " + num_word + " day of Christmas, my true love sent to me:"

  # Step 2: Sing the gift phrases
  by_day = dataset.set_index("Day")[phrase_col]
  gifts = ""
  for i in range(num, 1, -1):
    gifts = gifts + by_day.loc[i] + ",\n"
  if num == 1:
    gifts = gifts + by_day.loc[1] + "."
  else:
    gifts = gifts + "and " + by_day.loc[1] + "."

  # Step 3: Put it all together and return
  return intro + "\n" + gifts


In [58]:
song = sing_day(xmas, 12, "Full.Phrase")
print(song)

On the twelfth day of Christmas, my true love sent to me:
twelve drummers drumming,
eleven pipers piping,
ten lords a-leaping,
nine ladies dancing,
eight maids a-milking,
seven swans a-swimming,
six geese a-laying,
five golden rings,
four calling birds,
three french hens,
two turtle doves,
and a partridge in a pear tree.
